<a href="https://colab.research.google.com/github/Mounish2023/Build-LLMFromScratch/blob/main/reproduce_gpt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import GPT2LMHeadModel

In [ ]:
model_hf = GPT2LMHeadModel.from_pretrained('gpt2')
sd_hf = model_hf.state_dict()

for k, v in sd_hf.items():
    print(k,v.shape)

transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias 

In [ ]:
sd_hf["transformer.wpe.weight"].view(-1)[:20]

tensor([-0.0188, -0.1974,  0.0040,  0.0113,  0.0638, -0.1050,  0.0369, -0.1680,
        -0.0491, -0.0565, -0.0025,  0.0135, -0.0042,  0.0151,  0.0166, -0.1381,
        -0.0063, -0.0461,  0.0267, -0.2042])

In [ ]:
sd_hf["lm_head.weight"].view(-1)[:20]

tensor([-0.1101, -0.0393,  0.0331,  0.1338, -0.0485, -0.0789, -0.2398, -0.0895,
         0.0253, -0.1074, -0.1811, -0.0672,  0.0739, -0.0161,  0.0117,  0.1245,
        -0.0020, -0.0815,  0.0338,  0.2365])

In [ ]:
# # from ast import increment_lineno
# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.imshow(sd_hf["transformer.wpe.weight"], cmap="gray")

In [ ]:
# plt.plot(sd_hf["transformer.wpe.weight"][:,150])
# plt.plot(sd_hf["transformer.wpe.weight"][:,200])
# plt.plot(sd_hf["transformer.wpe.weight"][:,250])

# GPT2

1.  From the original transformer model, the encoder is removed, the middle self-attention + layernorm + residual connections are removed. the cross attention between the encoder and this middle layer of the block is not present in the gpt-2 layer.
2.  reshuffling of layernorms (it was moved to the input of each sub-block), similar to a pre-activation residual network
3. Additional layer normalization was added after the final self-attention block

In the original transformer model from the Attention all you need paper, pe are fixed. to sinosuids and cosines of different frequencies. but in gpt2 paper they are learned.

In [ ]:
# plt.imshow(sd_hf["transformer.h.1.attn.c_attn.weight"][:300,:300], cmap='gray')

In [ ]:
# from transformers import pipeline, set_seed
# generator = pipeline('text-generation',model='gpt2')
# set_seed(42)
# generator(text_inputs="Hi",max_length=30, num_return_sequences=5)

# Why multihead-attention and not singlehead

1. Computation is little easier on the gpu because in single head self-attention - you will have bigger Q,K,V and producing the output tensor for each token is compute heavy and moreover gpu's are good at parallel processing and we can divide these computations and concat them together at  the end.


# Is it better in terms of model performance, is it an optimization? Is it helping in generalization with some noise inducing.

The main reason why multihead self-attention exists is that each head learns a different subspace of attention. some might concentrate on the positions, some on punctuations, some on noun-verb referencing etc.,

It is not noise inducing like say dropout, but these reduce overfitting and act as a regularization techniques since they are learning diversed specialized patterns.

But i have this question - how can you be so sure it is learning different things and not the same thing multiple times. and also lets say we have kept a seed for the intialization at the start of training script. does this not mean the linear layers of Q,K,V are initialized the same way and will produce the same thing even when divided among different heads? explain in detail step by step. and help build intuition towards this.

1. firstly, even though we kept a same seed these objects in the heads are initialized independtly and not same unless the values are actually copied.
2. during training these subspaces of multiheads learn different things because of the gradients.
3. it is possible sometimes some of these heads tend to concentrate on certain aspects of the semantics and actually a wasted parameters. so head pruning techniques are also getting adopted.




1. Multilevel perceptron is same as Feedforward layer
2. remember the gradients are additive in nature and it is preferable to have free flow of these gradients from the supervision to the inputs. this residual pathway is important. so in the Block Module, we have additive for x


1. Attention is an aggregation function, it is pooling function, weighted function, its a reducer function
2. MLP or FFN (Feedforward Network) is on each individual token. they map it.

# Why GELU and not RELU for the gpt2

# because of Dead ReLU neuron problem.
the activations that are falling near zero at the flat layer does not give any gradients. but GELU always gives out certain gradients, local ones. And makes the NNs keep going.

BERT, gpt2, llama models picked this up.

gpt2 uses tanh approximation of GELU non-linearity function



In [ ]:
# from dataclasses import dataclass
# import torch
# import torch.nn as nn
# from torch.nn import functional as F
# import math



# class CausalSelfAttention(nn.Module):
#     def __init__(self, config):
#         super().__init__()
#         self.config = config
#         # Rather than having three separate layers for q,k,v - we have a single layer for efficiency
#         self.c_attn = nn.Linear(config.n_embd, 3* config.n_embd) #it is to make the x (B,T,C) into (B,T,3C)
#         self.n_head = config.n_head
#         self.n_embd = config.n_embd
#         self.c_proj = nn.Linear(config.n_embd, config.n_embd)
#         self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
#                              .view(1,1,config.block_size, config.block_size))


#     def forward(self, x):
#         B,T,C = x.shape

#         qkv = self.c_attn(x)
#         q,k,v = qkv.split(self.n_embd, dim=2) #break them up on dim=2 which is channels (embedding dimensions)
#         q = q.view(B,T, self.n_head, C//self.n_head ).transpose(1,2) #(B,nh,T,hs)
#         k = k.view(B,T, self.n_head, C//self.n_head ).transpose(1,2) #(B,nh,T,hs)
#         v = v.view(B,T, self.n_head, C//self.n_head ).transpose(1,2) #(B,nh,T,hs)

#         att = (q @ k.transpose(-2,-1)) * (1.0 / math.sqrt(k.size(-1))) #computing attention scores (affinity) (B,nh,T,T)
#         att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
#         att = F.softmax(att, dim=-1)
#         y = att @ v #(B,nh,T,T) @ (B,nh,T,hs) -> (B,nh,T,hs)
#         y = y.transpose(1,2).contiguous().view(B,T,C) #reassign all head outputs side by side
#         y = self.c_proj(y)
#         return y



# class MLP(nn.Module): #Multi-layer perceptron is same as feedforward network
#     def __init__(self, config):
#         super().__init__()

#         self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd) #c_fc — "Fully Connected"
#         self.gelu = nn.GELU(approximate='tanh')
#         self.c_proj = nn.Linear(4*config.n_embd, config.n_embd)  # c_proj — "Projection"

#     def forward(self, x):
#         x = self.c_fc(x)
#         x=self.gelu(x)
#         x = self.c_proj(x)
#         return x

# # class MLP(nn.Module): #Multi-layer perceptron is same as feedforward network
# #     def __init__(self, config):
# #         super().__init__()

# #         self.mlp = nn.Sequential(
# #             nn.Linear(config.n_embd, 4 * config.n_embd),
# #             nn.GELU(approximate='tanh'),
# #             nn.Linear(4*config.n_embd, config.n_embd)
# #         )


# #     def forward(self, x):
# #         x = self.mlp(x)
# #         return x


# class Block(nn.Module):
#     def __init__(self, config):
#         super().__init__()

#         self.ln_1 = nn.LayerNorm(config.n_embd)
#         self.attn = self.MultiHeadAttention(config)
#         self.ln_2 = nn.LayerNorm(config.n_embd)
#         self.mlp = MLP(config)

#     def forward(self, x):
#         x = x + self.attn(self.ln_1(x))
#         x = x + self.mlp(self.ln_2(x))

#         return x

# @dataclass
# class GPTConfig:
#     n_embd: int= 768
#     n_layer: int= 12
#     block_size: int = 1024
#     batch_size: int = 64
#     n_head: int = 12  # MultiHead attention layer. each head has different head sizes - n_embd/n_heads
#     vocab_size: int = 50257


# class GPT(nn.Module):
#     def __init__(self, config):
#         super().__init__()
#         self.config = config

#         self.transformer = nn.ModuleDict(
#             dict(
#                 wte = nn.Embedding(config.vocab_size, config.n_embd ),
#                 wpe = nn.Embedding(config.block_size, config.n_embd),
#                 h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
#                 ln_f = nn.LayerNorm(config.n_embd)
#             )
#         )
#         self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias= False)

#     def  forward(self, idx):
#         B,T = idx.shape
#         assert T <=self.config.block_size, f"Cannot forward sequence of length {T}, block size is {self.config.block_size}"

#         pos = torch.arange(0,T, dtype=torch.long, device=idx.device)
#         pos_emb = self.transformer.wpe(pos)
#         tok_emb = self.transformer.wte(idx)

#         x = tok_emb + pos_emb

#         for block in self.transformer.h:
#             x = block(x)

#         x = self.transformer.ln_f(x)

#         logits = self.lm_head(x)
#         return logits

#     @classmethod
#     def from_pretrained(cls, model_type):
#         """Loads pretrained GPT-2 model weights from huggingface"""
#         assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
#         from transformers import GPT2LMHeadModel
#         print("loading weights from pretrained gpt: %s" % model_type)

#         # n_layer, n_head and n_embd are determined from model_type
#         config_args = {
#             'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
#             'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
#             'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
#             'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
#         }[model_type]
#         config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
#         config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
#         # create a from-scratch initialized minGPT model
#         config = GPTConfig(**config_args)
#         model = GPT(config)
#         sd = model.state_dict()
#         sd_keys = sd.keys()
#         sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

#         # init a huggingface/transformers model
#         model_hf = GPT2LMHeadModel.from_pretrained(model_type)
#         sd_hf = model_hf.state_dict()

#         # copy while ensuring all of the parameters are aligned and match in names and shapes
#         sd_keys_hf = sd_hf.keys()
#         sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
#         sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
#         transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
#         # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
#         # this means that we have to transpose these weights when we import them
#         assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
#         for k in sd_keys_hf:
#             if any(k.endswith(w) for w in transposed):
#                 # special treatment for the Conv1D weights we need to transpose
#                 assert sd_hf[k].shape[::-1] == sd[k].shape
#                 with torch.no_grad():
#                     sd[k].copy_(sd_hf[k].t())
#             else:
#                 # vanilla copy over the other parameters
#                 assert sd_hf[k].shape == sd[k].shape
#                 with torch.no_grad():
#                     sd[k].copy_(sd_hf[k])

#         return model

# # num_return_sequences = 5
# # max_length = 30

# # model = GPT.from_pretrained('gpt2')
# # print("didn't crash yay!")

# # model.eval()
# # model.to('cuda')





# Created Model classes to import the Model parameters of gpt2
# Created DataLoaders that provides next batches for training.
#

# Tricky things
1. Weights initialization: Xavier std - 1/sqrt(dimension)
    a. Linear layers - bias
    b. Embedding layers
    c. Residual connections
    d. LayerNorm

Xavier initialization, also known as Glorot initialization, is a widely used method for initializing the weights of neural networks to help prevent the problems of vanishing or exploding gradients during training.

scale the activations to make sure that the std are within the range of 1 in the residual pathways during the forward pass. this scaling is done with 1/sqrt(N) (2 * n_layers).

2. Layer Normalization in transformers (across columns) / BatchNorm in other things (across samples)
3. weights of token embeddings and output linear head are same.
4. CausalSelfAttention - batches, number of heads, T, C (head size)

# Kernel fusion. model = torch.compile(model)

takes all of the code and makes it into one object for computation

# Increasing the speed of training.

1. reduce mantissa (precision) of the floating point numbers
FP32 -> TF32 ->BF16
if we go for FP16 -> we have to implement gradient scalers. for others, there is very little code change.
torch.set_float32_matmul_precision('high')

2. TF32 ->BF16 (pytorch converts some of the matrices to bfloat16, but some of it are still left at float32)
     
     with torch.autocast(device_type=device, dtype = torch.bfloat16):
        logits, loss = model(x,y)
the bottleneck here is still the memory. even though operations are happening very fast, these numbers need to be accessed and transported. it is time consuming process.

3. torch.compile(model)
with the benefit of knowing what are the operations that are going to happen before hand, it can reduce the number of roundtrips that need to happen between GPU and HBM. there is a memory hierarchical structure that needs to be understood to make the analysis better.

but even torch.compile doesn't do all of the operations in GPU - for example self attention layers.

4. Flash attention - all of the multiheaded causal self-attention operations are done with the help of kernel fusion.



